In [2]:
import spacy
spacy.load("en")
#print(spacy.load("en"))
from spacy.lang.en import English
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

import nltk
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))


nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

#def prepare_text_for_lda(text):
    #tokens = tokenize(text)
    #tokens = [token for token in tokens if len(token) > 4]
    #tokens = [token for token in tokens if token not in en_stop]
    #tokens = [get_lemma(token) for token in tokens]
    #return tokens

print('\n')
print('\n')
print('\n')
print()
import random
text_data = []
with open('dataset.csv') as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
        if random.random() > .99:
            #print(tokens)
            text_data.append(tokens)
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

import gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)
    print('\n')
    print('\n')
    print('\n')
    print('\n')
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/larissaba/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/larissaba/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!









(0, '0.054*"wireless" + 0.041*"system" + 0.028*"power" + 0.028*"battery"')








(1, '0.051*"network" + 0.035*"cross" + 0.019*"relational" + 0.019*"spell"')








(2, '0.049*"base" + 0.026*"digital" + 0.026*"measurement" + 0.026*"fpgas"')








(3, '0.042*"range" + 0.023*"structure" + 0.023*"document" + 0.023*"voltage"')








(4, '0.032*"dynamic" + 0.032*"network" + 0.017*"smart" + 0.017*"noise"')










/Users/larissaba/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
